In [1]:
from mas_automl.automl import AutoML

ImportError: cannot import name 'AutoML' from 'mas_automl.automl' (unknown location)

In [ ]:
train, test = train, test
start_time = time.perf_counter()
automl = AutoML(
    task='classification',
    use_preprocessing_pipeline=False,
    feature_selector_type=None,
    use_val_test_pipeline=False,
    auto_models_init_kwargs = {
        "metric": "roc_auc",
        "time_series": False,
        "models_list": ["all"],
        "blend": True,
        "stack": True
    },
    n_jobs=12, 
    random_state=0,
    )
automl = automl.fit(
    train.drop(columns=[target_name,]), train[target_name].to_numpy(), 
    test.drop(columns=[target_name,]), test[target_name].to_numpy(),
    auto_model_fit_kwargs = {
        "tuning_timeout": 10
    }
)
train_time = time.perf_counter() - start_time

test_predictions = automl.predict(test.drop(columns=[target_name,]))
predict_time = time.perf_counter() - start_time - train_time
all_time = train_time + predict_time

test_score = roc_auc_score(test[target_name], test_predictions[:, 1])